In [1]:
!pip install imbalanced-learn
!pip install xgboost


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\whovi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\whovi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# 1. 데이터 로드
data = pd.read_csv("/Users/whovi/Downloads/monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 2. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# 3. 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

# 4. 파이프라인 설정
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False))
])

# 5. 간단한 하이퍼파라미터 그리드 정의
param_grid = {
    'classifier__n_estimators': [600, 700],  # 트리 개수
    'classifier__max_depth': [15, 17],         # 트리의 최대 깊이
    'classifier__learning_rate': [0.1],
    'classifier__reg_lambda':[1.0, 1.5]      # 학습률
}

# 6. 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 7. GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# 8. 모델 학습
grid_search.fit(X_train, y_train)

# 9. 베스트 모델 추출
best_model = grid_search.best_estimator_
print("최적의 하이퍼파라미터: ", grid_search.best_params_)

# 검증 데이터 평가
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

# 10. 테스트 데이터 평가
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
print(f"테스트 데이터 정확도: {test_accuracy:.4f}")
print(f"테스트 데이터 F1 스코어: {test_f1:.4f}")

# 11. 분류 보고서 출력
print("\n테스트 데이터에 대한 분류 보고서:")
print(classification_report(y_test, y_test_pred))


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\whovi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [23:24:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


최적의 하이퍼파라미터:  {'classifier__learning_rate': 0.1, 'classifier__max_depth': 15, 'classifier__n_estimators': 700, 'classifier__reg_lambda': 1.5}
Validation Accuracy: 0.91
Validation F1 Score: 0.95
테스트 데이터 정확도: 0.9029
테스트 데이터 F1 스코어: 0.9437

테스트 데이터에 대한 분류 보고서:
              precision    recall  f1-score   support

           0       0.64      0.65      0.65       900
           1       0.94      0.94      0.94      5700

    accuracy                           0.90      6600
   macro avg       0.79      0.80      0.79      6600
weighted avg       0.90      0.90      0.90      6600

